In [2]:
# import library
import pandas as pd # for data manipulation

%pip install pyodide-http
import pyodide_http # for loading external files from http 
pyodide_http.patch_all() 

import matplotlib.pyplot as plt # for plottings 

# import data 
weather = pd.read_csv('https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/wetter.csv')
kiwo = pd.read_csv('https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/kiwo.csv')
sale = pd.read_csv('https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/umsatzdaten_gekuerzt.csv')

# merge data 
#sale_weather_kiwo = sale.merge(weather, on='Datum', how='outer', suffixes = ("_sale", "_weather")) \
#.merge(kiwo, on='Datum', how='outer', suffixes = ("", "_kiwo"))

sale_weather = pd.merge_ordered(sale, weather, on='Datum', suffixes = ("_sale", "_weather"), how='outer')
sale_weather_kiwo = pd.merge_ordered(sale_weather, kiwo, on='Datum', suffixes = ("", "kiwo"), how='outer')        

# sale_weather_kiwo.head()

# === DATE-BASED FEATURES - categorizing "datum" values ===
sale_weather_kiwo['Datum'] = pd.to_datetime(sale_weather_kiwo['Datum'], errors='coerce')
sale_weather_kiwo = sale_weather_kiwo.dropna(subset=['Datum'])  # Remove rows with missing date
sale_weather_kiwo = sale_weather_kiwo.sort_values(by='Datum') 

sale_weather_kiwo['DayOfWeek'] = sale_weather_kiwo['Datum'].dt.dayofweek
sale_weather_kiwo['Weekend'] = sale_weather_kiwo['DayOfWeek'].isin([5, 6]).astype(int)
sale_weather_kiwo['Month'] = sale_weather_kiwo['Datum'].dt.month
sale_weather_kiwo['Year'] = sale_weather_kiwo['Datum'].dt.year
sale_weather_kiwo['Week'] = sale_weather_kiwo['Datum'].dt.isocalendar().week
sale_weather_kiwo['Day'] = sale_weather_kiwo['Datum'].dt.day

sale_weather_kiwo['KielerWoche'] = sale_weather_kiwo['KielerWoche'].fillna(0).astype(int)

# === WEATHER-BASED FEATURES ===

# Categorize temperature
temp_bins = [-float("inf"), 0, 10, 20, 30, float("inf")]
temp_labels = ["very cold", "cold", "mild", "warm", "hot"]
sale_weather_kiwo['TemperatureCategory'] = pd.cut(sale_weather_kiwo['Temperatur'], bins=temp_bins, labels=temp_labels)

# Categorize cloud cover
cloud_bins = [-1, 2, 5, 8, float("inf")]
cloud_labels = ["clear", "partly cloudy", "cloudy", "overcast"]
sale_weather_kiwo['CloudCategory'] = pd.cut(sale_weather_kiwo['Bewoelkung'], bins=cloud_bins, labels=cloud_labels)

# Categorize wind speed
wind_bins = [-1, 10, 20, 30, float("inf")]
wind_labels = ["light", "moderate", "strong", "very strong"]
sale_weather_kiwo['WindCategory'] = pd.cut(sale_weather_kiwo['Windgeschwindigkeit'], bins=wind_bins, labels=wind_labels)

# === HOLIDAYS FEATURES === 
# Categorize holidays using 'holiday' library
%pip install holidays
import holidays
german_holidays = holidays.Germany(state='SH')  # SH = Schleswig-Holstein
sale_weather_kiwo['IsHoliday_lib'] = sale_weather_kiwo['Datum'].apply(
    lambda x: 1 if x.date() in german_holidays else 0
)
sale_weather_kiwo['HolidayName'] = sale_weather_kiwo['Datum'].apply(
    lambda x: german_holidays.get(x.date(), 'No Holiday')
)

# === SEASONS FEATURES: month-based and temperature as backup ===
def get_season_month_temp_backup(date, temp):
    """
    Define seasons based on month first, with temperature as backup/alternative
    
    Primary criteria (month-based):
    - Winter: months 12, 1, 2
    - Spring: months 3, 4, 5  
    - Summer: months 6, 7, 8
    - Autumn: months 9, 10, 11
    
    Backup criteria (temperature-based):
    - Winter: temp ≤ 5°C
    - Spring: temp 5-12°C
    - Summer: temp 12-18°C
    - Autumn: temp > 18°C
    """
    month = date.month
    
    # Primary: Month-based classification
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    
    # Backup: If somehow month is not in expected range, use temperature
    if not pd.isna(temp):
        if temp <= 5:
            return 'Winter'
        elif 5 < temp <= 12:
            return 'Spring'
        elif 12 < temp <= 18:
            return 'Summer'
        else:  # temp > 18
            return 'Autumn'
    
    # Final fallback
    return 'Unknown'

# Apply month-based season classification with temperature backup
sale_weather_kiwo['Season'] = sale_weather_kiwo.apply(
    lambda row: get_season_month_temp_backup(row['Datum'], row['Temperatur']), axis=1
)

# === TIME SERIES SPLIT ===
# Split your dataset into a training set from 01.07.2013 to 31.07.2017, a validation set from 01.08.2017 to 31.07.2018, and a test set from 01.08.2018 to 31.07.2019.
train_start = '2013-07-01'
train_end = '2017-07-31'
val_start = '2017-08-01'
val_end = '2018-07-31'
test_start = '2018-08-01'
test_end = '2019-07-31'

train_data = sale_weather_kiwo[(sale_weather_kiwo['Datum'] >= train_start) & (sale_weather_kiwo['Datum'] <= train_end)]
val_data = sale_weather_kiwo[(sale_weather_kiwo['Datum'] >= val_start) & (sale_weather_kiwo['Datum'] <= val_end)]
test_data = sale_weather_kiwo[(sale_weather_kiwo['Datum'] >= test_start) & (sale_weather_kiwo['Datum'] <= test_end)]

print("Rows:", len(train_data))
print("Rows:", len(val_data))
print("Rows:", len(test_data))

sale_weather_kiwo.head()

sale_weather_kiwo.to_csv("merged_sales_weather_kiwo_data.csv", index=False)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Rows: 7523
Rows: 1849
Rows: 352
